# Manual Curation
The goal of this notebook is to add all the observed functionality in R. opacus to the genome scale model.<br>
This includes the ability to grow with phenol, ..., and ... .

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json
import urllib
import cobra
import cplex
import os
import requests
import collections
import itertools

import cobra.test
from cobra.flux_analysis import gapfill

### Load genome-scale model
Get *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated.xml")
model

Name,ropacus_curated_with_5_deleted_reactions
Memory address,0x07f35bb2aa210
Number of metabolites,1581
Number of reactions,2380
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


### Add Ortho pathway metabolites to the model
Ortho pathway reactions:

| Reaction ID | Reaction description | In CarveMe Model? | Primary Reactant(s) | Primary Product(s) | gene product rule |
| --- | --- | --- | --- | --- | --- |
| R_EX_phenol_e | Phenol Exchange | False | phenol (e) | n/a | n/a |
| phenol_transport| Phenol Transport | False | phenol (e) | phenol (c) | n/a |
| PHEMOX | Phenol Monooxygenase | False | phenol | catechol | n/a |
| CATDOX | Catechol Dioxygenase | True | catechol_c | cis,cis-muconate | n/a |
| MUCCYCI | Muconate Cycloisomerase | True | cis,cis-muconate | muconolactone | n/a |
| MUCLI | Muconolactone D-isomerase | True | muconolactone | 5-oxo-4,5-dihydrofuran-2-acetate | n/a |
| OXOAEL | 3-oxoadipate enol-lactonase | True | 5-oxo-4,5-dihydrofuran-2-acetate | 3-oxoadipate | n/a |
| 3OADPCOAT | 3-oxoadipate CoA-transferase | True | 3-oxoadipate | 3-oxoadipyl-CoA | n/a | 
| 3OXCOAT | 3-oxoadipyl-CoA thiolase | True | 3-oxoadipyl-coA | Succinyl-CoA and Acetyl-CoA | n/a |


Ortho pathway metabolites

| Metabolite ID | Metabolite name | Metabolite formula | In CarveMe Model? | 
| --- | --- | --- | --- | 
| phenol_e | Phenol (e) | C6H6O | False |
| phenol_c | Phenol (c) | C6H6O | False | 
| catechol_c | Catechol | C6H6O2 | True | 
| ccmuac_c | Cis,cis-muconate | C6H4O4 | True | 
| mucl_c | Mucolactone | C6H5O4 | True | 
| 5odhf2a | 5-Oxo-4,5-dihydrofuran-2-acetate | C6H5O4 | True | 
| 3oxoadp_c | 3-Oxoadipate | C6H6O5 | True | 
| oxadpcoa_c | 3-Oxoadipyl-CoA | C27H37N7O20P3S | True | 

### Add Phenol metabolite and reactions to model
Add metabolites

In [3]:
# Add external phenol 
phenol_e = cobra.Metabolite(
    'phenol_e',
    formula='C6H6O',
    name='phenol',
    compartment='e')

# Add cytosol phenol
phenol_c = cobra.Metabolite(
    'phenol_c',
    formula='C6H6O',
    name='phenol',
    compartment='c')

Add reactions

In [4]:
phenol_exchange = cobra.Reaction('R_EX_phenol_e')
phenol_exchange.name = 'Phenol exchange'
phenol_exchange.lower_bound = -1000.  
phenol_exchange.upper_bound = 1000.  
phenol_exchange.add_metabolites({
    phenol_e: -1.0,
})

phenol_transport = cobra.Reaction('phenol_transport')
phenol_transport.name = 'Phenol exchange'
phenol_transport.lower_bound = -1000. 
phenol_transport.upper_bound = 1000. 
phenol_transport.add_metabolites({
    phenol_e: -1.0,
    phenol_c:  1.0,
}) 

# EC 1.14.13.244
phenol_monooxygenase = cobra.Reaction('PHEMOX')
phenol_monooxygenase.name = 'Phenol exchange'
phenol_monooxygenase.lower_bound = 0.
phenol_monooxygenase.upper_bound = 1000.  # This is the default
phenol_monooxygenase.add_metabolites({
    phenol_c: -1.0,
    model.metabolites.get_by_id('nadh_c'):   -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('nad_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
}) 


model.add_reactions([phenol_exchange])
model.add_reactions([phenol_transport])
model.add_reactions([phenol_monooxygenase])

In [17]:
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["R_EX_phenol_e"] = .77
    model.medium = medium
    print(model.medium)
    solution = model.optimize()
    print(solution)

{'EX_h2o_e': 10.0, 'EX_h_e': 10.0, 'EX_cl_e': 10.0, 'EX_pi_e': 10.0, 'EX_nh4_e': 10.0, 'EX_fe3_e': 10.0, 'EX_k_e': 10.0, 'EX_ca2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_mn2_e': 10.0, 'EX_cobalt2_e': 10.0, 'EX_zn2_e': 10.0, 'EX_cu2_e': 10.0, 'EX_o2_e': 10.0, 'EX_fe2_e': 10.0, 'EX_mobd_e': 10.0, 'EX_so4_e': 10.0, 'R_EX_phenol_e': 0.77}
<Solution 0.065 at 0x7f3674578d90>


In [ ]:
model.reactions.get_by_id('HADPCOADH3')

In [ ]:
model.metabolites.get_by_id('nadh_c')

In [ ]:

# Add catechol
catechol_c = cobra.Metabolite(
    'catechol_c',
    formula='C6H6O2',
    name='phenol',
    compartment='c')


In [ ]:
for m in model.metabolites:
    if 'muconate' in m.name:
        print(m.name, m)
        for r in m.reactions:
            print(r.name)
            print(r.reaction)
        print()

In [ ]:
for m in model.metabolites:
    if 'acet' in m.name:
        print(m.name, m)
        for r in m.reactions:
            print(r.name)
            print(r.reaction)
        print()

In [ ]:
model.metabolites.get_by_id('oxadpcoa_c')

In [ ]:
for r in model.reactions:
    if 'muconate' in r.name:
        print(r.name, r)

In [ ]:
for r in model.metabolites.get_by_id('glc__D_e').reactions:
    print(r.name, r)
    print(r)
    print(r.gene_reaction_rule)
    print()

In [ ]:
for m in model.metabolites:
    if 'catechol' in m.name:
        print(m.name)

In [ ]:
for r in model.reactions:
    if r.name.startswith('R_EX'):
        print(r)
        print(r.name)
        print(r.metabolites)
        print(r.gene_reaction_rule)
        print()